## Altmap Experiments
### Examine altmap behaviour an barbell networks + compare to Infomap


In [1]:
from matplotlib.ticker import (AutoMinorLocator)

from altmap.altmap_helpers.general import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# generate network
clique_size = 20
nodes_connection = 0
N = 2 * clique_size + nodes_connection # num nodes
G = nx.barbell_graph(clique_size, nodes_connection)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# compute altmap cost for different community setups
nodes_ids = list(range(1, N+1))
clique_half1_size = int(clique_size/2)
clique_half2_size = clique_size - clique_half1_size

# ground truth
labels = [1] * clique_size + [2] * clique_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Ground Truth Cost L = {cost}\n')

# 2 mixed cliques
labels = [1] * clique_half1_size + [2] * clique_half2_size + [1] * clique_half2_size + [2] * clique_half1_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Mixed Communities Cost L = {cost}\n')

# 4 mixed cliques
labels = [1] * clique_half1_size + [2] * clique_half2_size + [3] * clique_half1_size + [4] * clique_half2_size
communities = dict(zip(nodes_ids, labels))

cost = altmap_cost(G, communities)
print (f'Four Mixed Communities Cost L = {cost}\n')


Ground Truth Cost L = -0.9737686599571793

Mixed Communities Cost L = -0.0021921002339283513

Four Mixed Communities Cost L = -0.16604553583129333



In [3]:
# compute essential cost function values for a barbell network with
# given clique size
def barbell_cost(clique_size = 3, print_output=False):
    nc = clique_size
    m = nc*(nc-1)+1 # number of edges in the network
    p0 = (nc - 1) /(2*m) # stat prob for 'normal' nodes
    pc = nc / (2*m) # stat prob for the 2 connecting nodes

    J_ind = 2.0 * ((nc -2)*p0*np.log2(1.0-2*p0) + (p0+pc)*np.log2(1-(p0+pc)))
    J_true = np.log2(m) - 1.0 - (m-1) / m * np.log2(m-1)
    J_init = -1 -np.log2(m) + 1/m * ((m - nc) * np.log2(3*m - nc**2) + nc*np.log2(2*m-nc))
    J_lower_bound = (2*(nc - 1)*(nc - 1)/(2*m)*np.log2((nc - 1)/(2*m)) + 2*nc/(2*m)*np.log2(nc/(2*m)))
    if print_output:
        print (f"\nBarbell network with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
        print (f"Lower bound cost = {J_lower_bound}\n")
    
    return J_init, J_true, J_ind, J_lower_bound


nc_max = 30
nc_list = list(range(3, nc_max + 1))
J_init_list = np.zeros((len(nc_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_list = np.zeros_like(J_init_list)
J_lb_list = np.zeros_like(J_init_list)
J_altmap_list = np.zeros_like(J_init_list)
J_altmap_sci_list = np.zeros_like(J_init_list)
for i,nc in enumerate(nc_list):
    # compute analytical objectives
    J_init_list[i], J_true_list[i], J_ind_list[i], J_lb_list[i] = barbell_cost(nc)
    
    # compute actual objectives achieved by altmap
    G = nx.barbell_graph(nc, 0)
    G = nx.convert_node_labels_to_integers(G, first_label=1)
    communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True)
    J_altmap_list[i] = altmap_cost(G, communities_found)
    
    communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True, init='sc')
    J_altmap_sci_list[i] = altmap_cost(G, communities_found)
    
    

    

Spectral clustering finished in 0.0031810000000001004 seconds.
Spectral clustering finished in 0.002854000000000134 seconds.
Spectral clustering finished in 0.0031850000000002154 seconds.
Spectral clustering finished in 0.0035199999999999676 seconds.
Spectral clustering finished in 0.003330999999999751 seconds.
Spectral clustering finished in 0.0036079999999998336 seconds.
Spectral clustering finished in 0.043258999999999936 seconds.
Spectral clustering finished in 0.0301499999999999 seconds.
Spectral clustering finished in 0.04449999999999976 seconds.
Spectral clustering finished in 0.05149999999999988 seconds.
Spectral clustering finished in 0.017217000000000038 seconds.
Spectral clustering finished in 0.048976999999999826 seconds.
Spectral clustering finished in 0.0482680000000002 seconds.
Spectral clustering finished in 0.053815999999999864 seconds.
Spectral clustering finished in 0.0860840000000005 seconds.
Spectral clustering finished in 0.05492700000000017 seconds.
Spectral clus

In [4]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# plt.suptitle('Barbell network - objective over clique size')
ax.plot(nc_list, -J_true_list, 'x-', label='Ground truth')
ax.plot(nc_list, -J_init_list, '^--', label='Each node as a module')
ax.plot(nc_list, -J_ind_list, 'o--', label='Maximum independet sets')
# ax.plot(nc_list, -J_lb_list, 'o--', label='Upper bound')
ax.plot(nc_list, -J_altmap_sci_list, '.--', ms=8, lw=2, label='Implementation with SCI')
ax.plot(nc_list, -J_altmap_list, '.--', ms=8, lw=2, label='Implementation')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$', labelpad=20)
ax.set_ylabel('Synthesizing Infomap objective $\mathcal{J}(m)$', labelpad=20)
ax.legend()
plt.tight_layout()


In [2]:
# generate network
clique_size = 5
nodes_connection = 0
N = 2 * clique_size + nodes_connection # num nodes
G = nx.barbell_graph(clique_size, nodes_connection)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# run community detection
communities_found, num_communities_found,_ ,_ = infomap(G, altmap=False)
# communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True)
# communities_found, num_communities_found,_,_ = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

cost = altmap_cost(G, communities_found)
print (f'Achieved cost L = {cost}')


OrderedDict([(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2)])
We found 2 communities.
Achieved cost L = -0.7253784700009026


In [6]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)

In [7]:
# generate network
clique_size = 5
nodes_connection = 0
N = 2 * clique_size + nodes_connection # num nodes
G = nx.barbell_graph(clique_size, nodes_connection)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * clique_size + [2] * clique_size
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = list(range(1,clique_size+1)) * 2
communities_ind = dict(zip(nodes_ids, labels))


plt.close('all')

plt.figure(figsize=(5,8))
drawNetwork(G, communities_true)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_init)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_ind)
plt.tight_layout()


# plt.close('all')
# fig, axs = plt.subplots(1,3)
# fig.suptitle(f'Sample Barbell networks')
# 
# drawNetwork(G, communities_true, ax=axs[0])
# axs[0].set_xlabel('Ground truth')
# drawNetwork(G, communities_init, ax=axs[1])
# axs[1].set_xlabel('Initial partition')
# drawNetwork(G, communities_ind, ax=axs[2])
# axs[2].set_xlabel('Maximum independent sets')